# Importing the Required Excels

In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Side, Font
from openpyxl.utils import get_column_letter
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.styles.alignment import Alignment
import os

In [2]:
#######Configure inputs
cwd = os.getcwd()
demand_output = "28-04-2024_Demand_Automation.xlsx"
rmsku_master = r"03-05-2024 RMSKU Master List.xlsx"

nfc_os = os.path.join(cwd, "client_data", "NFC_OS.xlsx")
gfc_os = os.path.join(cwd, "client_data", "GFC_OS.xlsx")
kfc_os = os.path.join(cwd, "client_data", "KFC_OS.xlsx")

df_kfc=pd.read_excel(kfc_os, sheet_name='SOH RM', skiprows= 10)
df_gfc= pd.read_excel(gfc_os, skiprows=10)
df_nfc= pd.read_excel(nfc_os, skiprows=10)
df_rmsku= pd.read_excel(rmsku_master, skiprows=1)
df_rmsku = df_rmsku.iloc[:, 1:]

In [3]:
df_rmsku1=df_rmsku.copy()
df_rmsku1['FactCode']

0      GFC
1      GFC
2      GFC
3      GFC
4      GFC
      ... 
281    NFC
282    NFC
283    NFC
284    NFC
285    NFC
Name: FactCode, Length: 286, dtype: object

# Implementing the groupby and merges

In [4]:
#Creting RMSKU dataframes seperately for 3 Factode
df_rmsku_kfc= df_rmsku[df_rmsku['FactCode'] == 'KFC']
df_rmsku_gfc= df_rmsku[df_rmsku['FactCode'] == 'GFC']
df_rmsku_nfc= df_rmsku[df_rmsku['FactCode'] == 'NFC']
df_rmsku_gfc

#Changing the names of UOM present in file
replace_map = {
    'EACH': 'EA',
    'KILO GRAM': 'KGM',
    'METER': 'MTR',
    'ROLL': 'ROL',
    'CARTON': 'CRT',  
    'LITER': 'LIT'   
}
# Group by 'Item Number' and 'Unit Of Measure', then sum 'Quantity On Hand' for each group
df_gfc_new= df_gfc.groupby(['Item Number','Unit Of Measure'])['Quantity On Hand'].sum().reset_index()
# Rename columns
df_gfc_new.columns = ['RM Code','RM UOM', 'RM-M1OS']
# Replace values in 'RM UOM' column using replace_map
df_gfc_new['RM UOM'] = df_gfc_new['RM UOM'].replace(replace_map)
# Repeat the same process for other DataFrames
df_kfc_new= df_kfc.groupby(['Item Number','Unit Of Measure'])['Quantity On Hand'].sum().reset_index()
df_kfc_new.columns =['RM Code','RM UOM', 'RM-M1OS']
df_kfc_new['RM UOM'] = df_kfc_new['RM UOM'].replace(replace_map)


# df_kfc_pm_new= df_kfc_pm.groupby(['Item Number','Unit Of Measure'])['Quantity On Hand'].sum().reset_index()
# df_kfc_pm_new.columns =['RM Code','RM UOM', 'RM-M1OS']
# df_kfc_pm_new['RM UOM'] = df_kfc_pm_new['RM UOM'].replace(replace_map)
# Concatenate the KFC DataFrames of PM and RM
# df_kfc_merged=pd.concat([df_kfc_pm_new,df_kfc_new],axis=0 )
df_kfc_merged=pd.concat([df_kfc_new],axis=0 )

df_nfc_new= df_nfc.groupby(['Item Number','Unit Of Measure'])['Quantity On Hand'].sum().reset_index()
df_nfc_new.columns = ['RM Code','RM UOM', 'RM-M1OS']
df_nfc_new['RM UOM'] = df_nfc_new['RM UOM'].replace(replace_map)

#Merging the df with the same Factcode
df_merge_nfc= pd.merge(df_rmsku_nfc,df_nfc_new,on=['RM Code','RM UOM'], how='left')
df_merge_kfc= pd.merge(df_rmsku_kfc,df_kfc_merged,on=['RM Code','RM UOM'], how='left')

df_merge_gfc= pd.merge(df_rmsku_gfc,df_gfc_new,on=['RM Code','RM UOM'], how='left')
#Concatinating all dataframes in a single df
df_merge_final=pd.concat([df_merge_nfc,df_merge_kfc,df_merge_gfc],axis=0 )
df_merge_final['Retain_Flag']=1
#Adding Missing_Flag if RM-M1OS is null it will be flagged as o else 1
df_merge_final['Missing_Flag'] = np.where(df_merge_final['RM-M1OS'].notnull(), 1, 0)
#Drop the extra column
df_merge_final.drop(columns=['RM Desc'], inplace=True)
df_merge_final.fillna(0)

df_merge_final.loc[df_merge_final['RM-M1OS'].isnull(), 'RM-M1OS'] = 0
df_merge_final

,RM Code,RM UOM,FactCode,RM-M1OS,Retain_Flag,Missing_Flag
0,1001101002,KGM,NFC,1.296166e+06,1,1
1,1001101007,KGM,NFC,5.942160e+05,1,1
2,1001101034,KGM,NFC,0.000000e+00,1,0
3,1001101035,KGM,NFC,4.720000e+04,1,1
4,1001101043,KGM,NFC,7.110000e+03,1,1
...,...,...,...,...,...,...
25,2012114624,EA,GFC,3.230000e+04,1,1
26,2012114658,EA,GFC,2.326900e+05,1,1
27,2012114662,EA,GFC,0.000000e+00,1,0
28,2012115030,EA,GFC,0.000000e+00,1,0


# Formatting into Excel

In [5]:

# Write the name of dataframe and tab name that you want in excel
dataframes = {
    'RM-M1OS': df_merge_final,
   }

# Specify the path to save your Excel file
file_path = r'03-05-2024 RM-M1OS.xlsx'
# Using ExcelWriter to write each DataFrame to a separate sheet
with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    for sheet_name, dataframe in dataframes.items():
        dataframe.to_excel(writer, sheet_name=sheet_name, index=False)

    # Load the workbook and apply formatting
    workbook = writer.book
    for sheet_name in dataframes.keys():
        worksheet = workbook[sheet_name]
        worksheet.sheet_view.showGridLines = False  # Remove gridlines

        # Add one row and column at the top
        worksheet.insert_rows(1)
        worksheet.insert_cols(1)
        worksheet.cell(row=1, column=1)

        # Add table borders
        border = Border(left=Side(style='thin'), 
                        right=Side(style='thin'), 
                        top=Side(style='thin'), 
                        bottom=Side(style='thin'))
        for row in worksheet.iter_rows(min_row=2, min_col=2, max_col=worksheet.max_column):
            for cell in row:
                cell.border = border

        
# Set light blue color for headers
        for cell in worksheet.iter_cols(min_row=2, min_col=2):
            cell[0].fill = PatternFill(start_color="ADD8E6", end_color="ADD8E6", fill_type="solid")  # Light blue
    
        # Remove borders for the extra added row and column at the top
        for cell in worksheet['A1:C1'][0]:
            cell.border = None
# Autofit columns
        for col in worksheet.columns:
            max_length = 0
            column = col[0].column_letter  # Get the column name
            for cell in col:
                try:  # Necessary to avoid error on empty cells
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = (max_length + 2) * 1.2
            worksheet.column_dimensions[column].width = adjusted_width
 # Convert pixels to character units
        

        # Apply auto filters starting from the 2nd row in the B column
        #worksheet.auto_filter.ref = worksheet.dimensions
        last_row = worksheet.max_row
        last_column = worksheet.max_column
        range_str = f"B2:{get_column_letter(last_column)}{last_row}"
        worksheet.auto_filter.ref = range_str
# Save the Excel file
workbook.save(file_path)

print("All DataFrames are written to {}".format(file_path))


All DataFrames are written to 03-05-2024 RM-M1OS.xlsx
